# Importing corpus and reading it

In [212]:
# FULL DATASET 

from nltk.corpus import brown
train_lab = brown.tagged_sents()[0:len(brown.tagged_sents())*80/100]
test = brown.sents()[0:len(brown.sents())*20/100][11:12]
test_lab=brown.tagged_sents()[0:len(brown.tagged_sents())*20/100][11:12]


# Making the Data Structures

In [179]:
from itertools import tee, izip

transition_tri={}
transition_bi={}
context={}
emission={}

def window(iterable, size):
    iters = tee(iterable, size)
    for i in xrange(1, size):
        for each in iters[i:]:
            next(each, None)
    return izip(*iters)


# MAKING DICT {(TAG,TAG,TAG) : COUNT}
           

for i in train_lab:
    i=[("START","<s>")]+i
    i=i+[("END","</s>")]
    for each in window(i,3):
        temp=list(each)
        temp2=[]
        temp2.append(temp[0][1])
        temp2.append(temp[1][1])
        temp2.append(temp[2][1])
        temp=temp2
        for i in range(0,len(temp)):
            if "-" in temp[i]:
                temp[i]=temp[i].split("-")[0]
            if "+" in temp[i]:
                temp[i]=temp[i].split("+")[0]
        if transition_tri.has_key(tuple(temp)):
            transition_tri[tuple(temp)]+=1
        else:
            transition_tri[tuple(temp)]=1


            
# MAKING DICT {(TAG,TAG) : COUNT}


for i in train_lab:
    i=[("START","<s>")]+i
    i=i+[("END","</s>")]
    for each in window(i,2):
        temp=list(each)
        temp2=[]
        temp2.append(temp[0][1])
        temp2.append(temp[1][1])
        temp=temp2
        for i in range(0,len(temp)):
            if "-" in temp[i]:
                temp[i]=temp[i].split("-")[0]
            if "+" in temp[i]:
                temp[i]=temp[i].split("+")[0]
        if transition_bi.has_key(tuple(temp)):
            transition_bi[tuple(temp)]+=1
        else:
            transition_bi[tuple(temp)]=1
            

            
# MAKING CONTEXT {(TAG) : COUNT}


for i in train_lab:
    i=[("START","<s>")]+i
    for each in window(i,1):
        temp=list(each)
        temp2=[]
        temp2.append(temp[0][1])
        temp=temp2
        for i in range(0,len(temp)):
            if "-" in temp[i]:
                temp[i]=temp[i].split("-")[0]
            if "+" in temp[i]:
                temp[i]=temp[i].split("+")[0]
        if context.has_key(tuple(temp)):
            context[tuple(temp)]+=1
        else:
            context[tuple(temp)]=1


# MAKING EMISSION { (TAG,WORD) : COUNT }

N=0

for i in train_lab:
    i=[("START","<s>")]+i
    i=i+[("END","</s>")]
    for each in window(i,1):
        N+=1
        temp=list(each)
        temp2=[]
        temp2.append(temp[0][1])
        temp2.append((temp[0][0]).lower())
        temp=temp2
        for i in range(0,len(temp)):
            if "-" in temp[i]:
                temp[i]=temp[i].split("-")[0]
            if "+" in temp[i]:
                temp[i]=temp[i].split("+")[0]
        if emission.has_key(tuple(temp)):
            emission[tuple(temp)]+=1
        else:
            emission[tuple(temp)]=1
        
       
print context

{(u'BEM*',): 4, (u'HVN',): 185, (u'WP$',): 230, (u'HV*',): 16, (u'(',): 2341, (u'AP',): 8408, (u'PP$$',): 110, (u'PPLS',): 314, (u'PPL',): 913, (u'RN',): 7, (u'EX',): 1887, (u'DO*',): 292, (u'PN',): 1820, (u'PPS',): 13571, (u'NN$',): 1569, (u'BEDZ*',): 70, (u'PPSS',): 10485, (u'CD',): 13802, (u'',): 2782, (u'DTI',): 2581, (u')',): 2372, (u'VBG',): 14731, (u'DTX',): 85, (u'DT$',): 4, (u'.',): 48424, (u'CC',): 32177, (u'NPS',): 1215, (u'NP$',): 2253, (u'NN',): 146799, (u'TO',): 12732, (u'VBN',): 26374, (u'NR$',): 75, (u'CS',): 18871, (u'MD*',): 556, (u'BE',): 5724, (u'VB',): 28017, (u'NNS$',): 296, (u'WRB',): 3641, (u'DOZ*',): 63, (u"''",): 6119, (u'NP',): 33668, (u'WPS',): 3469, (u'NNS',): 52089, (u'QL',): 7547, (u'HVD*',): 46, (u'RBT',): 88, (u'NIL',): 157, (u'HV',): 3400, (u'HVZ',): 2360, (u'DOD',): 759, ('<s>',): 45872, (u'BEN',): 2058, (u'JJT',): 903, (u'RB$',): 8, (u'MD',): 10767, (u'RP',): 4289, (u'BER*',): 30, (u"'",): 250, (u'ABL',): 313, (u'BEG',): 593, (u'DT',): 7756, (u'WQL',

# Interpolation and Transition Probability Calculation

In [183]:
lam_1 = 0.90
lam_2 = 0.90
lam_3 = 0.90

def Transition_Prob(trig):
    if len(trig) > 2:                      # IT IS A TRIGRAM
        
        prev1 = trig[0]
        prev2 = trig[1]
        tag = trig[2]
        
        P1 = lam_1 * context[tuple([prev1])]/N + (1-lam_1)/N
        
        P2 = lam_2 * transition_bi.get(tuple((prev1+" "+prev2).split()),0)/(context[tuple([prev1])]+1)+(1-lam_2)*P1
        
        P3 = lam_3*transition_tri.get(tuple((prev1+" "+prev2+" "+tag).split()),0)/(transition_bi.get(tuple((prev1+" "+prev2).split()),0)+1)+(1-lam_3)*P2
        return P3
    else:                                            # ITS A BIGRAM
        prev = trig[0]
        tag  = trig[1]
        
        P1 = lam_1 * context[tuple([prev])]/N + (1-lam_1)/N
        
        P2 = lam_2 * transition_bi.get(tuple((prev+" "+tag).split()),0)/(context[tuple([prev])]+1)+(1-lam_2)*P1
        return P2


#    Emission Probability with some smoothing probability

In [184]:
lamda = 0.90

def Emission_Prob(tag2word):
    
    tag = tag2word[0]
    word = tag2word[1]
    
    prob = lamda * emission.get(tag2word,0)/context[tuple([tag])]+(1-lamda)/N
    
    return prob

# Forward Step

In [225]:
import math

def Fetch_Context(k):
    if k == -1 or k == 0:               # In case we are at beginning retun start
        return [tuple(["<s>"])];
    else:                               # For a word return full sentence
        return context


def forward_step(line):
    
    words=[]
    
    for i in line:
        words.append(i.lower())
    
    # NOW WE HAVE A LIST OF WORDS FOR A SENTENCE, ALL IN LOWER CASE
    
    l=len(words)
    
    best_score={}
    best_edge={}
    
    best_score[(0,"<s>","<s>")]=0     # init
    best_edge[(0,"<s>","<s>")]=None   # init
    
    for k in range(1,l+1):        # Travel in a sentence  
        
        for u in Fetch_Context(k-1):
            for v in Fetch_Context(k):
                for w in Fetch_Context(k-2):
                    
                    tri=tuple([w[0],u[0],v[0]])
                    if emission.get(tuple((v[0]+" "+words[k-1]).split()),0)!=0:
                        
                        score=best_score.get(tuple([k-1,w[0],u[0]]),0)+(-math.log(Transition_Prob(tri)))+(-math.log(Emission_Prob(tuple([v[0],words[k-1]]))))
                        
                        if best_score.get(tuple([k,u[0],v[0]]),0) == 0 or best_score.get(tuple([k,u[0],v[0]]),0) < score:
                            
                            best_score[tuple([k,u[0],v[0]])]=score
                            best_edge[tuple([k,u[0],v[0]])]=w[0]

    
    # NOW FOR 
    
    max_score = float('-Inf')
    
    u_max = None
    v_max = None
    
    for u in S(l-1):
        for v in S(l):
            
            tri=tuple([u[0],v[0],"</s>"])
            
            score=best_score.get(tuple([l,u[0],v[0]]),0)+(-math.log(Transition_Prob(tri)))
            
            if score > max_score:
                max_score = score
                u_max = u[0]
                v_max = v[0]
    
    tags = []
    tags.append(v_max)
    tags.append(u_max)
    
    for i, k in enumerate(range(l-2, 0, -1)):
        tags.append(best_edge.get(tuple([k+2,tags[i+1],tags[i]]),"NN"))
    tags.reverse()

    tagged_sentence = []
    for j in range(0, l):
        tagged_sentence.append((words[j],tags[j]))
    
    return tagged_sentence
      

def calcacc(dataset,lab_dataset):
    total=0
    for i in dataset:
        total+=len(i)
    
    correct =0
    
    for i in range(0,len(dataset)):
        x=forward_step(dataset[i])
        print x
        print lab_dataset[i]
        for j in range(0,len(x)):
            if x[j][1]==lab_dataset[i][j][1]:
                correct +=1
    print "ACCURACY"
    print "________________________________"
    print (correct*1.0/total)*100
    print "________________________________"
    print


calcacc(test,test_lab)

#print top 50 words emitted by tags

import operator

for tag in context:
    top_words={}
    for w in emission:
        t=w[0]
        word=w[1]
        if tag[0] == t:
            top_words[word]=emission[w]
    top_words = sorted(top_words.items(), key=operator.itemgetter(1),reverse=True)

    print tag,"\n",top_words[:50]
    

[(u'implementation', u'NN'), (u'of', u'NIL'), (u"georgia's", u'NP$'), (u'automobile', u'NN'), (u'title', u'NN'), (u'law', u'JJ'), (u'was', u'BEDZ'), (u'also', u'RB'), (u'recommended', u'VBD'), (u'by', u'VBZ'), (u'the', u'NIL'), (u'outgoing', u'JJ'), (u'jury', u'NN'), (u'.', u'.')]
[(u'Implementation', u'NN'), (u'of', u'IN'), (u"Georgia's", u'NP$'), (u'automobile', u'NN'), (u'title', u'NN'), (u'law', u'NN'), (u'was', u'BEDZ'), (u'also', u'RB'), (u'recommended', u'VBN'), (u'by', u'IN'), (u'the', u'AT'), (u'outgoing', u'JJ'), (u'jury', u'NN'), (u'.', u'.')]
ACCURACY
________________________________
64.2857142857
________________________________

(u'BEM*',) 
[(u"ain't", 4)]
(u'HVN',) 
[(u'had', 184), (u'hadd', 1)]
(u'WP$',) 
[(u'whose', 229), (u'whosever', 1)]
(u'HV*',) 
[(u"haven't", 14), (u"ain't", 2)]
(u'(',) 
[(u'(', 2340), (u'[', 1)]
(u'AP',) 
[(u'other', 1507), (u'many', 922), (u'more', 885), (u'same', 616), (u'last', 512), (u'few', 496), (u'most', 440), (u'much', 422), (u'several', 

(u'NNS$',) 
[(u"children's", 23), (u"women's", 22), (u"people's", 17), (u"men's", 14), (u"states'", 9), (u"years'", 8), (u"ladies'", 7), (u"artists'", 6), (u"months'", 5), (u"respondents'", 5), (u"parents'", 5), (u"writers'", 4), (u"mothers'", 4), (u"students'", 4), (u"farmers'", 4), (u"officers'", 3), (u"nations'", 3), (u"manufacturers'", 3), (u"janitors'", 3), (u"engineers'", 3), (u"boys'", 3), (u"motors'", 3), (u"dealers'", 3), (u"prisoners'", 3), (u"workers'", 3), (u"giants'", 2), (u"soldiers'", 2), (u"beginners'", 2), (u"employers'", 2), (u"brothers'", 2), (u"members'", 2), (u"horses'", 2), (u"planters'", 2), (u"trustees'", 2), (u"days'", 2), (u"citizens'", 2), (u"owners'", 2), (u"builders'", 2), (u"sisters'", 2), (u"taxpayers'", 2), (u"veterans'", 2), (u"queens'", 2), (u"portwatchers'", 1), (u"secretaries'", 1), (u"nurses'", 1), (u"composers'", 1), (u"smugglers'", 1), (u"lawyers'", 1), (u"pirates'", 1), (u"bishops'", 1)]
(u'WRB',) 
[(u'when', 1879), (u'where', 784), (u'how', 489)

(u'BED',) 
[(u'were', 2659)]
(u'VBZ',) 
[(u'seems', 238), (u'says', 184), (u'makes', 158), (u'comes', 128), (u'gives', 107), (u'becomes', 99), (u'means', 97), (u'knows', 93), (u'appears', 84), (u'provides', 81), (u'takes', 80), (u'goes', 75), (u'follows', 75), (u'shows', 73), (u'remains', 72), (u'looks', 59), (u'gets', 59), (u'needs', 55), (u'finds', 55), (u'requires', 54), (u'begins', 54), (u'wants', 53), (u'stands', 46), (u'depends', 46), (u'calls', 46), (u'includes', 45), (u'offers', 43), (u'consists', 42), (u'believes', 41), (u'feels', 41), (u'indicates', 40), (u'exists', 40), (u'writes', 40), (u'continues', 40), (u'involves', 39), (u'represents', 39), (u'holds', 39), (u'lies', 39), (u'brings', 38), (u'contains', 37), (u'serves', 37), (u'happens', 34), (u'tends', 34), (u'sees', 32), (u'meets', 32), (u'uses', 32), (u'increases', 30), (u'works', 30), (u'leads', 29), (u'suggests', 29)]
(u'CD$',) 
[(u"1960's", 3), (u"1961's", 2)]
(u'RB',) 
[(u'only', 1135), (u'also', 1010), (u'then', 1